 Данные опроса 4361 женщин из Ботсваны:

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [128]:
raw = pd.read_csv("h:\\MyProj\\Courcera\\Ml-mipt\\course4\\week3с\\work\\botswana.tsv", sep="\t", index_col=False) 
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [65]:
print(raw.religion.value_counts())

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64


In [79]:
#raw #.describe()
raw.loc[raw['agefm'].notnull() == False]

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
15,0,26,7,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,0.0,0.0
16,4,38,7,protestant,5.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,0.0,1.0
17,2,47,5,spirit,1.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4351,4,31,0,spirit,6.0,1.0,1.0,0,NaN,NaN,0,0.0,0.0,0.0,0.0
4352,4,35,3,spirit,6.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4356,0,16,7,protestant,2.0,1.0,0.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4357,2,28,7,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0


In [78]:
columns=["ceb","age","educ","religion","idlnchld","knowmeth","usemeth","evermarr","agefm","heduc","urban","electric","radio","tv","bicycle"]
df = raw.copy()
df.dropna(subset=columns, inplace=True)
df.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
count,1834.000000,1834.000000,1834.000000,1834.000000,1834.000000,1834.000000,1834.0,1834.000000,1834.000000,1834.000000,1834.000000,1834.000000,1834.000000,1834.000000
mean,3.665758,31.835333,5.230643,5.236096,0.966739,0.657579,1.0,20.696292,5.271538,0.541439,0.172846,0.709378,0.123228,0.304253
std,2.433189,7.689567,4.223918,2.371035,0.179365,0.474649,0.0,4.973461,4.821233,0.498416,0.378217,0.454173,0.328788,0.460216
min,0.000000,16.000000,0.000000,0.000000,0.000000,0.000000,1.0,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,26.000000,0.000000,4.000000,1.000000,0.000000,1.0,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,31.000000,6.000000,5.000000,1.000000,1.000000,1.0,20.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,5.000000,37.000000,7.000000,6.000000,1.000000,1.000000,1.0,23.000000,9.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,1.0,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Создайте признак nevermarr, равный единице там, где в agefm пропуски.
Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице XX будет мультиколлинеарность.
...

In [129]:
data = raw.copy()
data

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4357,2,28,7,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4358,4,24,5,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4359,1,26,0,spirit,5.0,1.0,0.0,1,22.0,7.0,0,0.0,1.0,0.0,0.0


In [130]:
data = raw.copy()
data.loc[data['agefm'].notnull(), 'nevermarr'] = 0
data.loc[data['agefm'].notnull() == False, 'nevermarr'] = 1
data['agefm'] = data['agefm'].fillna(0)
data.loc[(data['nevermarr'] == 1)&(data['heduc'].notnull() == False), "heduc"] = -1
data.columns.drop("evermarr")
data.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
count,4361.000000,4361.000000,4361.000000,4241.000000,4354.000000,4290.000000,4361.000000,4361.000000,4238.000000,4361.000000,4358.000000,4359.000000,4359.000000,4358.000000,4361.000000
mean,2.441642,27.405182,5.855996,4.615892,0.963252,0.577622,0.476726,9.861729,1.836008,0.516625,0.140202,0.701766,0.092911,0.275815,0.523274
std,2.406861,8.685233,3.927075,2.219303,0.188164,0.493996,0.499515,10.894991,4.475487,0.499781,0.347236,0.457535,0.290341,0.446975,0.499515
min,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,1.000000,19.000000,4.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [131]:
print("heduc null in ",4361-4238)
#data.loc[(data['nevermarr'] == 0)] #&(data['heduc'].notnull() == False)
#["heduc"] = -1 #
#data.loc[data.heduc == -1]
#print(data.heduc.value_counts())

heduc null in  123


In [132]:
data.loc[data['idlnchld'].notnull(), 'idlnchld_noans'] = 0
data.loc[data['idlnchld'].notnull() == False, 'idlnchld_noans'] = 1
data['idlnchld'] = data['idlnchld'].fillna(-1)

data.loc[data['heduc'].notnull(), 'heduc_noans'] = 0
data.loc[data['heduc'].notnull() == False, 'heduc_noans'] = 1
data['heduc'] = data['heduc'].fillna(-2)

data.loc[data['usemeth'].notnull(), 'usemeth_noans'] = 0
data.loc[data['usemeth'].notnull() == False, 'usemeth_noans'] = 1
data['usemeth'] = data['usemeth'].fillna(-1)


In [133]:
columns=["knowmeth", "electric", "radio", "tv", "bicycle"]
data.dropna(subset=columns, inplace=True)
data.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
count,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000,4348.000000
mean,2.437443,27.393514,5.863155,4.466191,0.963431,0.553818,0.476081,9.850736,1.730451,0.517249,0.140064,0.701702,0.092916,0.275299,0.523919,0.027139,0.028059,0.015179
std,2.401785,8.675630,3.922694,2.372879,0.187722,0.526808,0.499485,10.897246,4.459982,0.499760,0.347094,0.457564,0.290348,0.446716,0.499485,0.162507,0.165160,0.122280
min,0.000000,15.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,1.000000,19.000000,4.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [136]:
print("Cells: ",19*4348)

Cells:  82612


In [135]:
data

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4357,2,28,7,protestant,4.0,1.0,1.0,0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4358,4,24,5,protestant,4.0,1.0,1.0,0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4359,1,26,0,spirit,5.0,1.0,0.0,1,22.0,7.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + evermarr +'\
                    'agefm + heduc + urban + electric + radio + tv + bicycle + '\
                    'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'
            , data=data)
fitted = m1.fit(cov_type='HC1')
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     777.0
Date:                Sat, 25 Apr 2020   Prob (F-statistic):               0.00
Time:                        20:40:49   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [138]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [144]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + evermarr +'\
                    'agefm + heduc + urban + electric + bicycle + '\
                    'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'
            , data=data) 
fitted2 = m2.fit(cov_type='HC1') #
print(fitted2.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     1028.
Date:                Sat, 25 Apr 2020   Prob (F-statistic):               0.00
Time:                        20:41:01   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.4633      0.136    -10.

In [141]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [145]:
print("F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.467231, k1=5.000000


In [153]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + evermarr +'\
                    'agefm + heduc + urban + electric + bicycle + '\
                    'nevermarr + idlnchld_noans + heduc_noans'
            , data=data) 
fitted3 = m3.fit(cov_type='HC1') #cov_type='HC1'
print(fitted3.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     1026.
Date:                Sat, 25 Apr 2020   Prob (F-statistic):               0.00
Time:                        20:48:14   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.5829      0.137    -11.

In [155]:
#print("F=%f, p=%f, k1=%f" % m2.fit().compare_f_test(m3.fit()))
m2.fit().compare_f_test(m3.fit())

(92.89058230109713, 3.155200948040263e-40, 2.0)